# Step 2: Clean Network

This is a further post-processing function now found within the network_clean GOSTnets submodule. This function cleans the network by removing excessive nodes, and ensures all edges are bi-directional (except in the case of one-way roads).

WARNING: The Clean Network function is a computationally expensive function, so it may take a while to run. It outputs a pickled graph object, a dataframe of the edges, and a dataframe of the nodes. The expectation is that this will only have to be run once.

In [1]:
import os, sys
import time
import networkx as nx

In [2]:
# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
sys.path.append("../")
import GOSTnets as gn

In [3]:
pth = "./" # change this path to your working folder
data_pth = os.path.join(pth, 'tutorial_outputs')

# read back your graph from step 1 from you saved pickle
G = nx.read_gpickle(os.path.join(data_pth, 'iceland_unclean.pickle'))

## Optional step: Only use the largest sub-graph
Network analysis is often done on only connected graphs. Disconnected graphs can result in paths that cannot reach their destination. Also, you can evaluate how connected your network is and have the option of going back and making more edits.

In [4]:
# Identify only the largest graph

# compatible with NetworkX 2.4
list_of_subgraphs = list(G.subgraph(c).copy() for c in nx.strongly_connected_components(G))
max_graph = None
max_edges = 0
for i in list_of_subgraphs:
    if i.number_of_edges() > max_edges:
        max_edges = i.number_of_edges()
        max_graph = i

# set your graph equal to the largest sub-graph
G = max_graph

In [5]:
# inspect the resulting Graph
nodes = list(G.nodes(data=True))
edges = list(G.edges(data=True))
print(len(nodes))
print(nodes[0])
print(len(edges))
print(edges[0])

4393
(8195, {'x': -21.7997148, 'y': 64.1236937})
6040
(8195, 12350, {'Wkt': <shapely.geometry.linestring.LineString object at 0x12967e3d0>, 'id': 11872, 'infra_type': 'residential', 'osm_id': '166647087', 'key': 'edge_11872', 'length': 0.06203796196973288})


In [6]:
# To become familiar with the function read the doc string
gn.clean_network?

Set up some parameters for the CleanNetwork function

In [7]:
Iceland_UTMZ = {'init': 'epsg:32627'}

WGS = {'init': 'epsg:4326'} # do not adjust. OSM natively comes in ESPG 4326

Run the CleanNetwork Function. It will write the outputs in the specified wpath because we have verbose as True.

In [8]:
print('start: %s\n' % time.ctime())
gn.clean_network(G, wpath = data_pth, output_file_name = 'iceland_network', UTM = Iceland_UTMZ, WGS = {'init': 'epsg:4326'}, junctdist = 50, verbose = True)
print('\nend: %s' % time.ctime())
print('\n--- processing complete')

start: Wed Feb 19 14:29:53 2020



/Users/thomasgertin3/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj-2.4.2.post1-py3.7-macosx-10.9-x86_64.egg/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/Users/thomasgertin3/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj-2.4.2.post1-py3.7-macosx-10.9-x86_64.egg/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/Users/thomasgertin3/anaconda3/envs/gostnets2/lib/python3.7/site-packages/geopandas-0.6.2-py3.7.egg/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:4326'} != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


364
513
250
381
Edge reduction: 6040 to 381 (93 percent)

end: Wed Feb 19 14:30:00 2020

--- processing complete


At this point, our road network is fully prepped. 
Move on to Step 3 to see how we can use this network for some travel time analysis!